Reading "ImageNet Classification with Deep Convolutional Neural Networks" paper by Alex Krizhevsky, Ilya Sutskever and Geoffrey E. Hinton - https://proceedings.neurips.cc/paper_files/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf

AlexNet is trained on 2 GPUs but this implementation works with a CPU.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler